In [1]:
import pandas as pd

In [22]:
df = pd.read_csv('spa_counts.csv')

Block:2367 Str:GAAG
Block:2416 Str:AAAG

Animal: ['SAAgoat417']   Pop: 3
Block:2367 Str:GAAG ['snp34558-scaffold408-457530', 'snp34557-scaffold408-409448', 'snp34556-scaffold408-363242', 'snp34555-scaffold408-328360']
Animal: ['CHAgoat121']   Pop: 3
Block:2416 Str:AAAG ['snp11580-scaffold1423-1357387', 'snp11581-scaffold1423-1398778', 'snp11582-scaffold1423-1439118', 'snp11583-scaffold1423-1468834']

In [4]:
block_2367 = ['snp34558-scaffold408-457530', 'snp34557-scaffold408-409448', 'snp34556-scaffold408-363242', 'snp34555-scaffold408-328360']

block_2416 = ['snp11580-scaffold1423-1357387', 'snp11581-scaffold1423-1398778', 'snp11582-scaffold1423-1439118', 'snp11583-scaffold1423-1468834']

In [16]:
import pysam 

vcf = pysam.VariantFile('./examples/true_vcfs/DRZdivAlp1M_DRZdivA1m025_true_filtered.vcf')

In [11]:
for record in vcf:
    if record.id in block_2367:
        print(record.id, record.chrom, record.pos, record.ref, record.alts, record.samples['SAAgoat417']['GT'])

snp34558-scaffold408-457530 9 54250041 A ('G',) (1, 0)
snp34557-scaffold408-409448 9 54298123 A ('G',) (0, 0)
snp34556-scaffold408-363242 9 54344329 A ('G',) (0, 1)
snp34555-scaffold408-328360 9 54379211 A ('G',) (1, 0)


Str1: GAAG, Str2: AAGA

In [14]:
for record in vcf:
    if record.id in block_2416:
        print(record.id, record.chrom, record.pos, record.ref, record.alts, record.samples['CHAgoat121']['GT'])

snp11580-scaffold1423-1357387 9 68977978 A ('G',) (0, 0)
snp11581-scaffold1423-1398778 9 69019369 A ('G',) (0, 0)
snp11582-scaffold1423-1439118 9 69059709 A ('G',) (1, 0)
snp11583-scaffold1423-1468834 9 69089425 A ('G',) (0, 1)


Str1: AAGA, Str2: AAAG

In [17]:
for record in vcf:
    if record.id in block_2416:
        print(record.id, record.chrom, record.pos, record.ref, record.alts, record.samples['GSTgoat60']['GT'])

snp11580-scaffold1423-1357387 9 68977978 A ('G',) (1, 1)
snp11581-scaffold1423-1398778 9 69019369 A ('G',) (0, 0)
snp11582-scaffold1423-1439118 9 69059709 A ('G',) (0, 0)
snp11583-scaffold1423-1468834 9 69089425 A ('G',) (0, 1)


In [41]:
with open('diff_markers.txt', 'r') as f:
    lines = f.readlines()
    
markers = [x.split()[1] for x in lines] 

vcf = pysam.VariantFile('./examples/true_vcfs/DRZdivAlp1M_DRZdivA1m025_true.vcf')

animals = list(df['Animal_ID'])

In [43]:
def get_maf(record):
    mafs = [0, 0]
    for val in record.samples:
        if val not in animals:
            continue
        if record.samples[val]['GT'][0] != None:
            mafs[record.samples[val]['GT'][0]] += 1
        if record.samples[val]['GT'][1] != None:
            mafs[record.samples[val]['GT'][1]] += 1
    return min(mafs) / sum(mafs)

In [44]:
mafs = {}
for record in vcf:
    if record.id in markers:
        maf = get_maf(record)
        # print(record.id, maf)
        mafs[record.id] = maf

In [45]:
with open('diff_markers_maf.txt', 'w') as f:
    for i, l in enumerate(lines):
        if mafs[l.split()[1]] >= 0.025:
            f.write(f'{l[:-2]}\t\t{mafs[l.split()[1]]}\t*\n')        
        else:    
            f.write(f'{l[:-2]}\t\t{mafs[l.split()[1]]}\n')        